<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from scipy.sparse import lil_matrix
import edward as ed
from edward.models import Beta
import tensorflow as tf

In [2]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [3]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = 0
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

In [4]:
def infectedCascade(cascade,N=numNodes):
    A_inf = lil_matrix((N-1,N-1))
    
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for i in range(len(c_nodes)):
        for j in range(i):
            if c_times[j] < T:
                A_inf[(c_nodes[i],c_nodes[j])]=c_times[i]-c_times[j]
    
    return A_inf

In [5]:
def uninfectedCascade(cascade,N=numNodes):
    nodes = {s for s in range(N-1)}
    A_uninf = lil_matrix((N-1,N-1))
    
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for i in range(len(c_nodes)):
        for j in (nodes-set(c_nodes)):
            A_uninf[c_nodes[i],j]=T-c_times[i]
    
    return A_uninf

In [6]:
T=10
I = infectedCascade(v[0])
U = uninfectedCascade(v[0])

In [43]:
I_ph = tf.placeholder(tf.float32, [numNodes-1,numNodes-1])
U_ph = tf.placeholder(tf.float32, [numNodes-1,numNodes-1])
likelihood_ph = tf.placeholder(tf.float32, [None])

A = Beta(tf.ones([numNodes-1,numNodes-1]),tf.ones([numNodes-1,numNodes-1]))

psi_1 = -tf.reduce_sum(tf.multiply(A,I_ph))
psi_2 = -tf.reduce_sum(tf.multiply(tf.transpose(A),U_ph))
psi_3 = tf.reduce_sum(tf.multiply(tf.transpose(A),tf.sign(I_ph)))

log_prob = psi_1+psi_2+psi_3

In [44]:
data = {
    I_ph: I,
    U_ph: U}

In [270]:
inference = ed.MAP([A], data)

In [271]:
sess = ed.get_session()
tf.global_variables_initializer().run()

inference.run(n_iter=2500)

ValueError: setting an array element with a sequence.

In [45]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from edward.models.random_variable import RandomVariable
from tensorflow.contrib.distributions import Distribution

try:
    from tensorflow.contrib.distributions import FULLY_REPARAMETERIZED
except Exception as e:
    raise ImportError("{0}. Your TensorFlow version is not supported.".format(e))

class Cascade(RandomVariable, Distribution):
    def __init__(self,
                params,
                validate_args=False,
                allow_nan_stats=True,
                name="Cascade"):
        """Initialize an `Cascade` random variable.
        Args:
            params: tf.Tensor.
            Collection of samples. Its outer (left-most) dimension
            determines the number of samples.
        """
        parameters = locals()
        with tf.name_scope(name, values=[params]):
            with tf.control_dependencies([]):
                self._params = tf.identity(params, name="params")
                try:
                    self._n = tf.shape(self._params)[0]
                except ValueError:  # scalar params
                    self._n = tf.constant(1)
                    
    super(Cascade, self).__init__(
        dtype=self._params.dtype,
        reparameterization_type=FULLY_REPARAMETERIZED,
        validate_args=validate_args,
        allow_nan_stats=allow_nan_stats,
        parameters=parameters,
        graph_parents=[self._params, self._n],
        name=name)
    
    @staticmethod
    def _param_shapes(sample_shape):
        return {"params": tf.convert_to_tensor(sample_shape, dtype=tf.int32)}

    @property
    def params(self):
        """Distribution parameter."""
        return self._params

    @property
    def n(self):
        """Number of samples."""
        return self._n
    
    def _log_prob(self, value):
        # TODO
        # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/distributions/python/ops/distribution.py#L183
        raise NotImplementedError("log_prob is not implemented")

    def _sample_n(self, n, seed=None):
        input_tensor = self.params
        if len(input_tensor.shape) == 0:
            input_tensor = tf.expand_dims(input_tensor, 0)
            multiples = tf.concat([tf.expand_dims(n, 0), [1] * len(self.event_shape)], 0)
            return tf.tile(input_tensor, multiples)
        else:
            probs = tf.ones([self.n]) / tf.cast(self.n, dtype=tf.float32)
            cat = tf.contrib.distributions.Categorical(probs)
            indices = cat._sample_n(n, seed)
            tensor = tf.gather(input_tensor, indices)
            return tensor
        
        

In [9]:
import numpy as np
np.array(v[0])

array([753.      ,   0.      , 261.      ,   1.35072 , 704.      ,
         3.966973, 145.      ,   4.897442,  73.      ,   5.925426,
       286.      ,   6.630252, 556.      ,   7.843829,  80.      ,
         8.141659, 458.      ,   8.775306,   8.      ,   8.788488,
       467.      ,   8.902549, 554.      ,   8.908081,  53.      ,
         9.237536,   1.      ,   9.451033, 584.      ,   9.591078,
       843.      ,   9.602514, 116.      ,   9.780086, 130.      ,
         9.81525 , 545.      ,   9.828353, 579.      ,   9.970419,
       535.      ,   9.984903])